### Simulated Annealing in Julia for the Number Partitioning Problem

#### Problem set and initialising initial state

In [69]:
# Problem set to partition
numbers = [4, 2, 7, 1]

# Initialising a random state of spins = {-1, 1}
state_0 = [Int(rand(Int64(0):Int64(1))) for i = 1:size(numbers,1)]
for i in 1:size(numbers,1)
    if state_0[i] == 0
       state_0[i] -= 1
    end
end

# Number of iterations of SA (terminating control)
iterations = 1000;

#### Neighbour state function

In [70]:
function neighbour(s::Vector{Int64}) # Moves the current state to a neighbouring state   
    s_proposal = s
    
    # Keeps the first spin value constant to remove double degeneracy of solutions 
    # From the paper “Ising formulations of many NP problems” [Lucas]
    for i in 2:size(s, 1)
        random = Int64(rand(Int64(0):Int64(1)))
        if s[i] == -1
            s_proposal[i] += random
            if random == 1
                s_proposal[i] += 1       
            end                
        else
            s_proposal[i] -= random
            if random == 1
                s_proposal[i] -= 1
            end                    
        end
    end
    return s_proposal
end

neighbour (generic function with 1 method)

#### Cost function 

In [71]:
# Calculates cost function according to Ising formulation
function cost(numbers, state)
    cost_root = 0
    for i in 1:size(state, 1)
        cost_root += numbers[i]*state[i] 
    end
    cost = cost_root^2
    return cost
end

cost (generic function with 1 method)

#### Cooling schedules

In [78]:
# Defines multiple cooling schedule functions 

function log_temp(i)
    return 1 / (1 + log(i + 1))
end

function constant_temp(i)
    return 1
end

function exp_multiplicative_cooling(i)
    alpha = rand(Float64(0.8):Float64(0.9))
    return alpha^i
end

function log_multiplicative_cooling(i)
    alpha = rand(Int64(1):Int64(100))
    return 1 / (1 + alpha * log(i + 1))
end

function linear_multiplicative_cooling(i)
    alpha = rand(Int64(0):Int64(100))
    return 1 / (1 + alpha * i)
end

function quadratic_multiplicative_cooling(i)
    alpha = rand(Int64(0):Int64(100))
    return 1 / (1 + alpha * i^2)
end

quadratic_multiplicative_cooling (generic function with 1 method)

#### Simulated annealing function

In [77]:
function simulated_annealing(state_0::Vector{Int64},
                             numbers::Vector{Int64},
                             iterations::Int64,
                             cost::Function,
                             neighbour::Function,
                             temperature::Function,
                             keep_best::Bool,
                             trace::Bool)
    
    state = state_0                 # Initialises state to initial state
    best_state = state_0            # Initialises best state to initial state
    x = copy(numbers)      
    flag = 0
    temp = 10                       # Sets initial temperature
    
    for i = 1:iterations
        
        temp *= temperature(i)      # Calls the cooling schedule function
        
        state_temporary_1 = copy(state)
        state_temporary_2 = copy(state)
        y = cost(x, state_temporary_1)
        
        # Breaks out of the loop if cost = 0 is reached
        if  y == 0
            best_state = state
            flag = 1
            if keep_best
                println()
                println("problem : $x")
                println("solution: $best_state")
                println("cost    : $y")
            break
            end
        end
        
        if trace println("$i: state = $state") end
        if trace println("$i: y = $y") end
        
        # Finds the neighbouring state
        state_n = neighbour(state_temporary_2)
        y_n = cost(x, state_n)
        
        # Breaks out of the loop if cost = 0 is reached
        if  y_n == 0
            best_state = state_n
            flag = 1
            if keep_best
                println()
                println("problem : $x")
                println("solution: $best_state")
                println("cost    : $y_n")
            break
            end
        end
        
        if trace println("$i: state_n = $state_n") end
        if trace println("$i: y_n = $y_n") end
        
        # Compares the values of cost function of state and neighbouring state
        if y_n <= y
            state = state_n
            if trace println("accept state") end
        else
            # Calculates probability of worse move
            probability = exp( - ((y_n - y) / temp))              
            if trace println("$i: probability = $probability") end
            
            # Checks if probability is less than a random number produced in the range -> (0, 1)
            if rand() <= probability
                state = state_n
                if trace println("accept state") end
            else
                state = state
                if trace println("reject state") end
            end
        end
    
        # Compares values of cost function for best state
        if cost(x, state) < cost(x, best_state)
            best_state = state
        end
    
        if trace println() end
    end
  
    if flag == 0
        println()
        if keep_best
            println("problem : $x")
            println("solution: $best_state")
        else
            println("problem : $x")
            println("solution: $state")
        end
    end
end

simulated_annealing (generic function with 1 method)

#### Solution to the number partitioning problem

In [80]:
solution = simulated_annealing(state_0,     # initial state of spins = {-1, 1} (length of problem set)
                               numbers,     # problem set to be partitioned
                               iterations,  # number of iterations of SA (only termination condition)
                               cost,        # cost function derived from the Ising 
                               neighbour,   # function that moves the state to a neighbouring state
                               log_temp,    # cooling schedule
                               true,        # choice to keep the best state  
                               true)        # choice to trace the SA steps

1: state = [-1, -1, -1, -1]
1: y = 196
1: state_n = [-1, 1, 1, 1]
1: y_n = 36
accept state

2: state = [-1, 1, 1, 1]
2: y = 36
2: state_n = [-1, 1, 1, 1]
2: y_n = 36
accept state

3: state = [-1, 1, 1, 1]
3: y = 36
3: state_n = [-1, 1, -1, 1]
3: y_n = 64
3: probability = 4.888529073535828e-11
reject state

4: state = [-1, 1, 1, 1]
4: y = 36
4: state_n = [-1, 1, -1, -1]
4: y_n = 100
4: probability = 3.175631896780674e-62
reject state

5: state = [-1, 1, 1, 1]
5: y = 36
5: state_n = [-1, -1, -1, -1]
5: y_n = 196
5: probability = 0.0
reject state

6: state = [-1, 1, 1, 1]
6: y = 36

problem : [4, 2, 7, 1]
solution: [-1, -1, 1, -1]
cost    : 0
